## A demo using Hydrogen Hamiltonian with GPT-QE.

In [36]:
import torch
from gqe.mingpt.utils import set_seed

set_seed(3407)

In [37]:
from qwrapper.operator import PauliObservable
from gqe.mingpt.cost import EnergyCost
from qswift.compiler import DefaultOperatorPool
from benchmark.molecule import DiatomicMolecularHamiltonian
from gqe.operator_pool.uccsd import UCCSD, do_generate_molecule
from gqe.common.initializer import HFStateInitializer
from gqe.util import get_device
from gqe.mingpt.callback import DefaultCallback, PrintMonitor, FileMonitor

# molecule = generate_molecule("Li", "H", 1.596, "sto-3g", bravyi_kitaev=False)
bond_length = 4.0
geometry = f"H 0.0 0.0 0.0\n" + f"Be 0.0 0.0 {bond_length}\n" + f"H 0.0 0.0 {2 * bond_length}\n"
molecule = do_generate_molecule(geometry, "sto-3g", bravyi_kitaev=False)
nqubit = 12

# prepare Hamiltonian
hamiltonian = DiatomicMolecularHamiltonian(nqubit, molecule, bravyi_kitaev=False)

# prepare operator_pool
uccsd = UCCSD(nqubit, molecule)
paulis = uccsd.paulis
paulis.append(PauliObservable("IIIIIIIIII"))
print('paulis', paulis)
num_operators = len(paulis)
initializer = HFStateInitializer(n_electrons=4)
pool = DefaultOperatorPool(paulis)
cost = EnergyCost(hamiltonian, initializer, pool,
                  [1/160, -1/160, 1/80, -1/80, 1/40, -1/40, 0.05, -0.05, 0.1, -0.1, 0.2, -0.2])


converged SCF energy = -14.9345493818237
paulis [+IIIXIIIIIIII, +IIIXZYIIIIII, +IIIXZZZYIIII, +IIIXZZZZZYII, +IIIXZZZZZZZY, +IIIYZXIIIIII, +IIIYZZZXIIII, +IIIYZZZZZXII, +IIIYZZZZZZZX, +IIXIIIIIIIII, +IIXXIIIIIIXY, +IIXXIIIIIIYX, +IIXXIIIIXYII, +IIXXIIIIYXII, +IIXXIIXYIIII, +IIXXIIYXIIII, +IIXXXYIIIIII, +IIXXYXIIIIII, +IIXYIIIIIIXX, +IIXYIIIIIIYY, +IIXYIIIIXXII, +IIXYIIIIYYII, +IIXYIIXXIIII, +IIXYIIYYIIII, +IIXYXXIIIIII, +IIXYYYIIIIII, +IIXZYIIIIIII, +IIXZZZYIIIII, +IIXZZZZZYIII, +IIXZZZZZZZYI, +IIYXIIIIIIXX, +IIYXIIIIIIYY, +IIYXIIIIXXII, +IIYXIIIIYYII, +IIYXIIXXIIII, +IIYXIIYYIIII, +IIYXXXIIIIII, +IIYXYYIIIIII, +IIYYIIIIIIXY, +IIYYIIIIIIYX, +IIYYIIIIXYII, +IIYYIIIIYXII, +IIYYIIXYIIII, +IIYYIIYXIIII, +IIYYXYIIIIII, +IIYYYXIIIIII, +IIYZXIIIIIII, +IIYZZZXIIIII, +IIYZZZZZXIII, +IIYZZZZZZZXI, +IXIIIIIIIIII, +IXXIIXZZZZYI, +IXXIIYZZZZXI, +IXXIXZZZZZZY, +IXXIYZZZZZZX, +IXYIIXZZZZXI, +IXYIIYZZZZYI, +IXYIXZZZZZZX, +IXYIYZZZZZZY, +IXZXIXZZZZZY, +IXZXIYZZZZZX, +IXZYIXZZZZZX, +IXZYIYZZZZZY, +IXZZZ

## FCI energy by diagonalization

In [38]:
from qwrapper.hamiltonian import compute_ground_state

print(compute_ground_state(hamiltonian))

-15.33649467289809


In [39]:
print("hf state:", hamiltonian.exact_value(initializer.init_circuit(12, [], "qulacs")))

hf state: -14.934549381823707


## Setup for GPT

In [49]:
# create a GPT instance
from gqe.mingpt.model import GPT

model_config = GPT.get_default_config()
model_config.model_type = 'gpt2'
model_config.vocab_size = cost.vocab_size()
model_config.n_gates = 30  # The number of gates for each circuit
model_config.block_size = model_config.n_gates
model_config.temperature = 5  # Each gate is generated with probability exp(-temperature * logit)
model_config.embd_pdrop = 0.1
model_config.resid_pdrop = 0.1
model_config.attn_pdrop = 0.1
model_config.std = 0.02
model_config.energy_offset = 14
model = GPT(model_config, cost)

number of parameters: 86.45M


In [50]:
# create a Trainer object
from gqe.mingpt.trainer import Trainer

train_config = Trainer.get_default_config()
train_config.learning_rate = 5e-7  # the model we're using is so small that we can go a bit faster
train_config.max_iters = 1000
train_config.num_workers = 10
train_config.n_samples = 50
trainer = Trainer(train_config, model)

running on device mps


In [51]:
file_monitor = FileMonitor()
callback_generator = DefaultCallback(model, monitors=[PrintMonitor(), file_monitor], del_temperature=0.01)
trainer.set_callback('on_batch_end', callback_generator.generate())
trainer.run()
torch.save(model.state_dict(), '../saved_models/gptqe_test_2')

iter_dt 0.00s; iter 0: train loss 0.12835 temperature: 5
mean_logits tensor([-14.8512, -14.9562, -15.2267, -14.9710, -14.7714, -14.8837, -14.9010,
        -14.7092, -14.8180, -15.0351, -14.8493, -14.9278, -14.8723, -14.8471,
        -14.8040, -14.9433, -14.8794, -14.7996, -14.9195, -14.8049, -14.9153,
        -14.8954, -14.8289, -15.0263, -14.9498, -14.7140, -14.9564, -14.9862,
        -14.9376, -14.7848, -14.9624, -14.8172, -14.7612, -14.9951, -14.9789,
        -14.8185, -15.0089, -14.9130, -14.9163, -14.8289, -14.8306, -14.8919,
        -14.8468, -14.7122, -14.9324, -15.0341, -15.0719, -14.8158, -14.8112,
        -14.8262], device='mps:0', grad_fn=<SubBackward0>)
energies: tensor([-14.8637, -14.9821, -14.8633, -14.6012, -14.8186, -14.8436, -14.7361,
        -14.8003, -14.8578, -14.9762, -14.7379, -14.8163, -14.9125, -14.9274,
        -14.8580, -14.7611, -14.7163, -14.7490, -14.8684, -14.8992, -14.7147,
        -14.9244, -14.6826, -14.8088, -14.8167, -14.7516, -14.7682, -14.8097,
    

KeyboardInterrupt: 

In [ ]:
print(file_monitor.min_energy)


In [ ]:
cost.sequence.tool = "qiskit"
print(file_monitor.min_indices)
cost.sequence._get_circuit(file_monitor.min_indices).qc.draw(output="mpl", plot_barriers=True)